<a href="https://colab.research.google.com/github/jbrouboni/Chalmers-Introduction-to-data-science-and-AI/blob/main/Assignment4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#DAT405 Introduction to Data Science and AI 
##2020-2021, Reading Period 2
## Assignment 4: Spam classification using Naïve Bayes 
There will be an overall grade for this assignment. To get a pass grade (grade 5), you need to pass items 1-3 below. To receive higher grades, finish items 4 and 5 as well. 

The exercise takes place in a notebook environment where you can chose to use Jupyter or Google Colabs. We recommend you use Google Colabs as it will facilitate remote group-work and makes the assignment less technical. 
Hints:
You can execute certain linux shell commands by prefixing the command with `!`. You can insert Markdown cells and code cells. The first you can use for documenting and explaining your results the second you can use writing code snippets that execute the tasks required.  

In this assignment you will implement a Naïve Bayes classifier in Python that will classify emails into spam and non-spam (“ham”) classes.  Your program should be able to train on a given set of spam and “ham” datasets. 
You will work with the datasets available at https://spamassassin.apache.org/old/publiccorpus/. There are three types of files in this location: 
-	easy-ham: non-spam messages typically quite easy to differentiate from spam messages. 
-	hard-ham: non-spam messages more difficult to differentiate 
-	spam: spam messages 

**Execute the cell below to download and extract the data into the environment of the notebook -- it will take a few seconds.** If you chose to use Jupyter notebooks you will have to run the commands in the cell below on your local computer, with Windows you can use 7zip (https://www.7-zip.org/download.html) to decompress the data.



In [1]:
#Download and extract data
!wget https://spamassassin.apache.org/old/publiccorpus/20021010_easy_ham.tar.bz2
!wget https://spamassassin.apache.org/old/publiccorpus/20021010_hard_ham.tar.bz2
!wget https://spamassassin.apache.org/old/publiccorpus/20021010_spam.tar.bz2
!tar -xjf 20021010_easy_ham.tar.bz2
!tar -xjf 20021010_hard_ham.tar.bz2
!tar -xjf 20021010_spam.tar.bz2

--2020-12-01 20:42:16--  https://spamassassin.apache.org/old/publiccorpus/20021010_easy_ham.tar.bz2
Resolving spamassassin.apache.org (spamassassin.apache.org)... 40.79.78.1, 95.216.24.32, 95.216.26.30, ...
Connecting to spamassassin.apache.org (spamassassin.apache.org)|40.79.78.1|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1677144 (1.6M) [application/x-bzip2]
Saving to: ‘20021010_easy_ham.tar.bz2.1’

20021010_easy_ham.t 100%[===================>]   1.60M  --.-KB/s    in 0.1s    

2020-12-01 20:42:16 (15.8 MB/s) - ‘20021010_easy_ham.tar.bz2.1’ saved [1677144/1677144]

--2020-12-01 20:42:16--  https://spamassassin.apache.org/old/publiccorpus/20021010_hard_ham.tar.bz2
Resolving spamassassin.apache.org (spamassassin.apache.org)... 40.79.78.1, 95.216.24.32, 95.216.26.30, ...
Connecting to spamassassin.apache.org (spamassassin.apache.org)|40.79.78.1|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1021126 (997K) [application/x-bzip2]
Sa

*The* data is now in the three folders `easy_ham`, `hard_ham`, and `spam`.

In [ ]:
!ls -lah

total 4.0M
drwxr-xr-x 1 root root 4.0K Dec  1 13:26 .
drwxr-xr-x 1 root root 4.0K Dec  1 13:24 ..
-rw-r--r-- 1 root root 1.6M Jun 29  2004 20021010_easy_ham.tar.bz2
-rw-r--r-- 1 root root 998K Dec 16  2004 20021010_hard_ham.tar.bz2
-rw-r--r-- 1 root root 1.2M Jun 29  2004 20021010_spam.tar.bz2
drwxr-xr-x 1 root root 4.0K Nov 20 17:15 .config
drwx--x--x 2  500  500 168K Oct 10  2002 easy_ham
drwx--x--x 2 1000 1000  20K Dec 16  2004 hard_ham
drwxr-xr-x 1 root root 4.0K Nov 13 17:33 sample_data
drwxr-xr-x 2  500  500  36K Oct 10  2002 spam


###1. Preprocessing: 
1.	Note that the email files contain a lot of extra information, besides the actual message. Ignore that for now and run on the entire text. Further down (in the higher-grade part), you will be asked to filter out the headers and footers. 
2.	We don’t want to train and test on the same data. Split the spam and the ham datasets in a training set and a test set. (`hamtrain`, `spamtrain`, `hamtest`, and `spamtest`)


In [1]:
import os
import email
import email.policy
import nltk 
import numpy as np
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd



In [2]:
#pre-processing code here
PATH=os.getcwd() #change it 
EASY_HAM_DIR = os.path.join(PATH, "easy_ham")
HARD_HAM_DIR = os.path.join(PATH, "hard_ham")
SPAM_DIR = os.path.join(PATH, "spam")
easy_ham_filenames = [name for name in sorted(os.listdir(EASY_HAM_DIR))]
hard_ham_filenames = [name for name in sorted(os.listdir(HARD_HAM_DIR))]
spam_filenames = [name for name in sorted(os.listdir(SPAM_DIR))]

print(len(easy_ham_filenames))
print(len(hard_ham_filenames))
print(len(spam_filenames))

#Loading email in spam,easy and hard ham!

def load_email(is_spam, filename, spam_path=PATH):
    directory = "spam" if is_spam else "easy_ham"
    with open(os.path.join(spam_path, directory, filename), "rb") as f:
        return email.parser.BytesParser(policy=email.policy.default).parse(f)
def load_hard_email(is_spam, filename, spam_path=PATH):
    directory = "spam" if is_spam else "hard_ham"
    with open(os.path.join(spam_path, directory, filename), "rb") as f:
        return email.parser.BytesParser(policy=email.policy.default).parse(f)

easy_ham_emails = [load_email(is_spam=False, filename=name) for name in easy_ham_filenames]
hard_ham_emails = [load_hard_email(is_spam=False, filename=name) for name in hard_ham_filenames]
spam_emails = [load_email(is_spam=True, filename=name) for name in spam_filenames]

def email_to_text(email):
    for part in email.walk():
        ctype = part.get_content_type()
        if not ctype in ("text/plain", "text/html"):
            continue
        try:
            content = part.get_content()
        except: # in case of encoding issues
            content = str(part.get_payload())
        if ctype == "text/plain":
            return content
        else:
            html = content

for i in range(len(easy_ham_emails)):
  easy_ham_emails[i]=email_to_text(easy_ham_emails[i])
for i in range(len(hard_ham_emails)):
  hard_ham_emails[i]=email_to_text(hard_ham_emails[i])
for i in range(len(spam_emails)):
  spam_emails[i]=email_to_text(spam_emails[i])


easy_ham = [] 
for val in easy_ham_emails: 
  if (val != None):
    easy_ham.append(val)  

hard_ham = [] 
for val in hard_ham_emails: 
  if (val != None):
    hard_ham.append(val)  

spam = []
for val in spam_emails: 
  if (val != None):
    spam.append(val) 

print(len(easy_ham))
print(len(hard_ham))
print(len(spam)) 

2551
250
501
2551
128
292


In [ ]:
hamdataset=np.vstack([easy_ham+hard_ham,np.zeros(2679)]).T
hamdataset[:,0:1].shape

In [4]:
hamdataset=np.vstack([easy_ham+hard_ham,np.zeros(2679)]).T
spamdataset=np.vstack([spam,np.ones(292)]).T


dataset=easy_ham+hard_ham+spam
target=np.zeros(2971)
target[2679:2971] = 1
whole_dataset=np.vstack([dataset,target]).T

whole_dataset.shape
df=pd.DataFrame(whole_dataset,columns =['Mail','Class']) 
#df
vectorizer = CountVectorizer()
counts = vectorizer.fit_transform(df['Mail'].values)
targets=df['Class'].values

#hamtrain, hamtest, target_train, target_train_test = train_test_split( hamdataset[0:1],hamdataset[:,2], test_size=0.7, random_state=42)





NameError: ignored

In [71]:
clfMNB=MultinomialNB()
clfMNB.fit(counts,targets)

clfMNB=MultinomialNB()
clfMNB.fit(counts,targets)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [22]:
vectorizer = CountVectorizer()
vector = vectorizer.fit_transform(easy_ham)
vectorizer2 = CountVectorizer()
vector2 = vectorizer2.fit_transform(hard_ham)
vectorizer3 = CountVectorizer()
vector3 = vectorizer.fit_transform(spam)

print(vector.shape)
print(vector2.shape)
print(vector3.shape)

(2551, 29222)
(128, 18706)
(292, 14940)


###2. Write a Python program that: 
1.	Uses four datasets (`hamtrain`, `spamtrain`, `hamtest`, and `spamtest`) 
2.	Trains a Naïve Bayes classifier (e.g. Sklearn) on `hamtrain` and `spamtrain`, that classifies the test sets and reports True Positive and False Negative rates on the `hamtest` and `spamtest` datasets. You can use `CountVectorizer` to transform the email texts into vectors. Please note that there are different types of Naïve Bayes Classifier in SKlearn ([Documentation here](https://scikit-learn.org/stable/modules/naive_bayes.html)). Test two of these classifiers that are well suited for this problem
- Multinomial Naive Bayes  
- Bernoulli Naive Bayes. 

Please inspect the documentation to ensure input to the classifiers is appropriate. Discuss the differences between these two classifiers. 





1) Creation of the four datasets(hamtrain, spamtrain, hamtest, and spamtest)

In [25]:
#hamtrain,hamtest,spamtrain,spamtest,  = train_test_split(easy_ham_data,target, test_size=0.3, random_state=111)

#tfidf_transformer = TfidfTransformer()
#X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
#X_train_tfidf.shape


ValueError: ignored

Your discussion here

### 3.Run your program on 
-	Spam versus easy-ham 
-	Spam versus hard-ham.

In [ ]:
#Code to report results here

###4.	To avoid classification based on common and uninformative words it is common to filter these out. 

**a.** Argue why this may be useful. Try finding the words that are too common/uncommon in the dataset. 

**b.** Use the parameters in Sklearn’s `CountVectorizer` to filter out these words. Update the program from point 3 and run it on your data and report your results.

You have two options to do this in Sklearn: either using the words found in part (a) or letting Sklearn do it for you. Argue for your decision-making.


###5. Eeking out further performance
Filter out the headers and footers of the emails before you run on them. The format may vary somewhat between emails, which can make this a bit tricky, so perfect filtering is not required. Run your program again and answer the following questions: 
-	Does the result improve from 3 and 4? 
- The split of the data set into a training set and a test set can lead to very skewed results. Why is this, and do you have suggestions on remedies? 
- What do you expect would happen if your training set were mostly spam messages while your test set were mostly ham messages? 

Re-estimate your classifier using `fit_prior` parameter set to `false`, and answer the following questions:
- What does this parameter mean?
- How does this alter the predictions? Discuss why or why not.

### What to report and how to hand in.

- You will need to clearly report all results in the notebook in a clear and appropriate way, either using plots or code output (f.x. "print statements"). 
- The notebook must be reproducible, that means, we must be able to use the `Run all` function from the `Runtime` menu and reproduce all your results. **Please check this before handing in.** 
- Save the notebook and share a link to the notebook (Press share in upper left corner, and use `Get link` option. **Please make sure to allow all with the link to open and edit.**
- Edits made after submission deadline will be ignored, graders will recover the last saved version before deadline from the revisions history.
- **Please make sure all cells are executed and all the output is clearly readable/visible to anybody opening the notebook.**